# Tweet Sentiment Analysis

[Data Source](https://data.world/crowdflower/brands-and-product-emotions)

## Imports

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# set a random seed for reproducibility
seed = 3490

## Load Data

In [2]:
encodings = ["utf-8-sig", "cp1252", "iso-8859-1", "latin1"]
for encoding in encodings:
    try:
        data = pd.read_csv('data/judge-1377884607_tweet_product_company.csv', encoding=encoding)
    except:
        continue
print(encoding)
data.head()

latin1


,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


## Explore Data

In [40]:
print(f'How much data do we have?\n{data.shape[0]} records.')
print('\n')
print(f'Example of a tweet in the dataset:\n"{data.tweet_text[0]}"')
print('\n')
print(f'Which products are the tweets concerning?\n{list(data.emotion_in_tweet_is_directed_at.value_counts().index)}')
print('\n')
class_distribution = data.is_there_an_emotion_directed_at_a_brand_or_product.value_counts(normalize=True)
print(f'Is this an imbalanced classification problem?\n{class_distribution}')

How much data do we have?
9093 records.


Example of a tweet in the dataset:
".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead!  I need to upgrade. Plugin stations at #SXSW."


Which products are the tweets concerning?
['iPad', 'Apple', 'iPad or iPhone App', 'Google', 'iPhone', 'Other Google product or service', 'Android App', 'Android', 'Other Apple product or service']


Is this an imbalanced classification problem?
No emotion toward brand or product    0.592654
Positive emotion                      0.327505
Negative emotion                      0.062686
I can't tell                          0.017156
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: float64


If we start by building a classifier which classifies tweets as containing positive or negative emotions, then we can only use 38% of the available data. For that reason, I want to build a multi-class classifier from the beginning.

## Prepare Data for Modeling

**Handle Missing Data**

Check for missing data.

In [12]:
data.isna().sum()

tweet_text                                               1
emotion_in_tweet_is_directed_at                       5802
is_there_an_emotion_directed_at_a_brand_or_product       0
dtype: int64

For now, we will ignore towards which product the sentiment in the tweet is directed, which allows us to ignore the missing data in the "emotion_in_tweet_is_directed_at" column.

View the record which is missing "tweet_text" data.

In [15]:
data.loc[data.tweet_text == None]

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product


We can safely drop this record.

In [3]:
clean_data = data.dropna(subset=['tweet_text'])

**Drop Data for Which Sentiment Category is "I can't tell"**

In [4]:
clean_data = clean_data.loc[clean_data.is_there_an_emotion_directed_at_a_brand_or_product != "I can't tell"]

**Drop Column Specifying At Which Product the Sentiment is Directed**

In [5]:
clean_data = clean_data.drop(columns='emotion_in_tweet_is_directed_at')

**Rename Sentiment Column**

In [6]:
clean_data = clean_data.rename({'is_there_an_emotion_directed_at_a_brand_or_product': 'sentiment'}, axis=1)

**Inspect First Few Rows of Cleaned Data:**

In [7]:
clean_data.head()

,tweet_text,sentiment
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Positive emotion


In [8]:
clean_data.shape

(8936, 2)

## Load Pre-trained GloVe Vectors

Load pre-trained word vectors (to give more context to our 9000 or so tweets.)

In [10]:
with open('data/glove.twitter.27B.50d.txt') as f:
    lines = f.readlines()

In [11]:
glove = dict()

for line in lines:
    items = line.split()
    word = items[0]
    vector = items[1:]
    glove[word] = [float(component) for component in vector]

Save to file, so we don't have to run the above code every time we restart the kernel.

In [13]:
import json
with open("data/glove.txt", "w") as filepath:
    json.dump(glove, filepath) 

**Load preprocessed glove dictionary here:**

In [9]:
import json
with open("data/glove.txt", "r") as file:
    glove = json.load(file)

In [10]:
len(glove.keys())

1193514

We have 1,193,514 words and characters which have been vectorized for us (in the context of Twitter.)

## Train Test Split

The next step is to split our data into training and testing sets, and then preprocess the tweet text for our classifier using our glove vectors.

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(clean_data['tweet_text'],
                                                    clean_data['sentiment'],
                                                    test_size=0.2,
                                                    random_state=seed)

X_train.head(2)

8841    Next #SXSW we're heading to is about designing...
1099    Talked to some great developers at the Android...
Name: tweet_text, dtype: object

## Vectorize Tweets for Modeling

Define a function to go through the text of each tweet and get the (GloVe) vector representation for each word, then mind the mean word vector for the entire tweet.

In [13]:
import string

def get_mean_word_vector(tweet):
    tokens = tweet.split()
    tweet_vectors = []
    for token in tokens:
        try:
            tweet_vectors.append(glove[token.lower().strip(string.punctuation)])
        except:
#             print(token)     we're losing quite a few tokens, but will ignore for now
            continue
    return np.mean(np.array(tweet_vectors), axis=0)

Note that this function does not pad each tweet. I am curious to see the effect of padding versus not padding on the classifier.

In [14]:
X_train_preprocessed = X_train.apply(get_mean_word_vector)

In [15]:
X_test_preprocessed = X_test.apply(get_mean_word_vector)

The series above are not in the correct format for a scikit learn classifier. To convert to a matrix of features, use the below loop:

In [16]:
preprocessed_values = dict()

for i in range(50):
    label = f'feature_{i}'
    values = [vector[i] for vector in X_train_preprocessed.values]
    preprocessed_values[label] = values

final_X_train = pd.DataFrame(preprocessed_values, index=X_train.index)

In [17]:
preprocessed_values = dict()

for i in range(50):
    label = f'feature_{i}'
    values = [vector[i] for vector in X_test_preprocessed.values]
    preprocessed_values[label] = values

final_X_test = pd.DataFrame(preprocessed_values, index=X_test.index)

## Label Encode Target

Label encode the target.

In [18]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

y_train_preprocessed = le.fit_transform(y_train)
y_test_preprocessed = le.transform(y_test)

print(le.classes_)

['Negative emotion' 'No emotion toward brand or product'
 'Positive emotion']


## Store Modeling Results

To keep up with the performance of different models, build a dictionary for scores.

In [19]:
model_eval = {'name': [], 'model': [], 'accuracy': []}

from sklearn.base import clone # to store models

## Build Baseline Classifier

Options for classifier:

- logistic regression
- decision tree
- k nearest neighbors
- random forest
- boosted random forest

Our baseline classifier will be a logistic regression.

In [20]:
from sklearn.linear_model import LogisticRegression

For logistic regression, the data needs to be scaled.

In [21]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

scaled_X_train = ss.fit_transform(final_X_train)
scaled_X_test = ss.transform(final_X_test)

Perform cross validation on the scaled training data.

In [22]:
from sklearn.model_selection import cross_val_score

In [23]:
baseline = LogisticRegression(random_state=seed)

cross_val_score(baseline, scaled_X_train, y_train_preprocessed)

array([0.64615385, 0.63706294, 0.63916084, 0.6550035 , 0.64660602])

Store the results.

In [24]:
model_eval['name'].append('baseline')
model_eval['model'].append(clone(baseline))
model_eval['accuracy'].append(np.mean(cross_val_score(baseline, scaled_X_train, y_train_preprocessed)))
pd.DataFrame(model_eval)

,name,model,accuracy
0,baseline,LogisticRegression(random_state=3490),0.644797


## Test Padding Mean Word Vector for Different Length Tweets

In [28]:
X_train.map(lambda x: len(x.split())).describe()

count    7148.000000
mean       17.758254
std         4.954223
min         2.000000
25%        14.000000
50%        18.000000
75%        21.000000
max        32.000000
Name: tweet_text, dtype: float64

Based on the above descriptive statistics for the length (in words) of each tweet, 21 would be a good "length" for the mean tweet vector.

\*Note: the mean tweet vector has a dimensionality of 50, but I am going to try considering a list of word vectors of length 21, regardless of whether a tweet has fewer or greater than 21 words in it.

In [37]:
test_array = [[1, 2, 5], [3, 4, 7]]
test_array.append([0]*3)
np.array(test_array)

array([[1, 2, 5],
       [3, 4, 7],
       [0, 0, 0]])

In [39]:
import string

def get_mean_word_vector2(tweet):
    tokens = tweet.split()
    tweet_vectors = []
    for token in tokens:
        try:
            tweet_vectors.append(glove[token.lower().strip(string.punctuation)])
        except:
#             print(token)     we're losing quite a few tokens, but will ignore for now
            continue
    # pad with 0 vectors if less than 21 words in length
    while len(tweet_vectors) < 21:
        tweet_vectors.append([0]*50)
    # save first 21 word vectors as an array
    padded_tweet_vector = np.array(tweet_vectors[:21])
    # return the mean of these 21 word vectors
    return np.mean(padded_tweet_vector, axis=0)

In [40]:
X_train_preprocessed2 = X_train.apply(get_mean_word_vector2)

X_test_preprocessed2 = X_test.apply(get_mean_word_vector2)

In [45]:
preprocessed_values = dict()

for i in range(50):
    label = f'feature_{i}'
    values = [vector[i] for vector in X_train_preprocessed.values]
    preprocessed_values[label] = values

final_X_train2 = pd.DataFrame(preprocessed_values, index=X_train.index)

preprocessed_values = dict()

for i in range(50):
    label = f'feature_{i}'
    values = [vector[i] for vector in X_test_preprocessed.values]
    preprocessed_values[label] = values

final_X_test2 = pd.DataFrame(preprocessed_values, index=X_test.index)

In [47]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

scaled_X_train2 = ss.fit_transform(final_X_train2)
scaled_X_test2 = ss.transform(final_X_test2)

In [48]:
cross_val_score(baseline, scaled_X_train2, y_train_preprocessed)

array([0.64615385, 0.63706294, 0.63916084, 0.6550035 , 0.64660602])

In [49]:
model_eval['name'].append('baseline_w_padded_mean_word_vector')
model_eval['model'].append(clone(baseline))
model_eval['accuracy'].append(np.mean(cross_val_score(baseline, scaled_X_train2, y_train_preprocessed)))
pd.DataFrame(model_eval)

,name,model,accuracy
0,baseline,LogisticRegression(random_state=3490),0.644797
1,baseline_w_padded_mean_word_vector,LogisticRegression(random_state=3490),0.644797


There is practically no difference, so I will continue without padding.